In [ ]:
### Imports des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns


#  Importation des données


In [ ]:
import pandas as pd
import numpy as np
from math import sqrt, log

# chargement des données
parkinson_DF = pd.read_csv('train_data.txt', sep = ',', header = None)

# Renommage des colonnes 
parkinson_DF.columns = ['SubjectId', 
                        'Jitter_LOCAL', 'Jitter_LOCAL_ABSOLUTE', 'Jitter_RAP','Jitter_PPQ5','Jitter_DDP', 
                        'Shimmer_LOCAL','Shimmer_LOCAL_DB','Shimmer_APQ3','Shimmer_APQ5','Shimmer_APQ11','Shimmer_DDA', 
                        'AC','NTH','HTN', 
                        'Median pitch','Mean pitch','Standard deviation','Minimum pitch','Maximum pitch',
                        'Number of pulses','Number of periods','Mean period','Standard deviation of period',
                        'Fraction of locally unvoiced frames','Number of voice breaks','Degree of voice breaks',
                        'UPDRS',
                        'Class Information']

#affichage de quelques tuples
parkinson_DF.head()


In [ ]:
parkinson_DF.shape


<h2>Attribute Information:</h2>
  -- 1. SubjectId: Entier qui identifie de manière unique chaque sujet<br>   
  
  -- 2. Jitter_LOCAL:  mesure de variation de la fréquence fondamentale (Paramètres de fréquence)<br>
  
  -- 3. Jitter_LOCAL_ABSOLUTE: mesure de variation de la fréquence fondamentale absolue  (Paramètres de fréquence)<br>
  
  -- 4. Jitter_RAP: mesure de variation de la fréquence fondamentale (Paramètres de fréquence=Relative Amplitude Perturbation) <br>
  
  -- 5. Jitter_PPQ5: mesure de variation de la fréquence fondamentale (Paramètres de fréquence= five-point Period Perturbation Quotient )<br>
  
  -- 6. Jitter_DDP: mesure de variation de la fréquence fondamentale (Paramètres de fréquence=Average absolute difference of differences between cycles, divided by the average period)<br>
  
  -- 7. Shimmer_LOCAL: Mesure de variation dans l'amplitude (Amplitude parameter)<br>   
  
  -- 8. Shimmer_LOCAL_DB: Mesure de variation dans l'amplitude en decibels (Amplitude parameter)<br>
  
  -- 9. Shimmer_APQ3: Mesure de variation dans l'amplitude (Amplitude parameter= Three point Amplitude Perturbation Quotient)<br>
  
  -- 10. Shimmer_APQ5: Mesure de variation dans l'amplitude (Amplitude parameter=Five point Amplitude Perturbation Quotient)<br>
  
  -- 11. Shimmer_APQ11: Mesure de variation dans l'amplitude (Amplitude parameter=11-point Amplitude Perturbation Quotient)<br>
  
  -- 12. Shimmer_DDA: Mesure de variation dans l'amplitude (Amplitude parameter=Average absolute difference between consecutive differences between the amplitudes of consecutive periods )<br>
  
  -- 13. AC: mesure du rapport entre le bruit et les composantes tonales de la voix <br>
  
  -- 14. NTH: mesure du rapport entre le bruit et les composantes tonales de la voix(Noise-to-Harmonics Ratio)<br>
  
  -- 15. HTN: mesure du rapport entre le bruit et les composantes tonales de la voix(Harmonics-to-Noise Ratio)<br>
  
  -- 16. Median pitch: la mediane de pitch<br> 
  -- 17. Mean pitch: le pas (pitch) moyen<br>
  -- 18. Standard deviation: Écart-type de pitch<br>
  -- 19. Minimum pitch: le pitch minimale<br>
  -- 20. Maximum pitch: le pitch maximale<br>
  -- 21. Number of pulses: nombre d'impulsions  <br>
  -- 22. Number of periods: nombre de periodes des impulsions<br>
  -- 23. Mean period: periode moyenne des impulsions<br>
  -- 24. Standard deviation of period: écart type des impulsions<br>
  -- 25. Fraction of locally unvoiced frames: parametre de voix <br>
  -- 26. Number of voice breaks: nombre de voix cassés (parametre de voix)<br>
  -- 27. Degree of voice breaks: degre de voix cassé (parametre de voix)<br>
  -- 28. UPDRS: Unified Parkinsons Disease Rating Scale score pour chaque patient<br>
  -- 29. Class Information<br>

                        

# Analyse des données

<h5>Verification d'existence de valeurs nulles dans notre dataset</h5>

In [ ]:
plt.figure(figsize=(12,4))
sns.heatmap(parkinson_DF.isnull(),cbar=False,cmap='viridis',yticklabels=False)
plt.title('Missing value in the dataset')


D'apres ce plot on vois tres bien qu'il y'a pas de valeurs manquantes dans notre dataset parkinson

<h5>Distribution of Parkinson Speak Disease</h5>

In [ ]:
#Valeur de class 1= malade sinon 0= non malade
malade = (parkinson_DF['Class Information'] == 1)
print("Nomre de gens atteintes de Parkinson Disease:",malade.sum())
nonMalade = (parkinson_DF['Class Information'] == 0)
print("Nomre de gens non atteintes de Parkinson Disease:",nonMalade.sum())

<h4>Matrice de corrélation:</h4>
Pour afficher la corrélation entre les differente données

In [ ]:
import seaborn as sns
corr_matrix = parkinson_DF.corr()

plt.figure(figsize=(20,12))
sns.heatmap(corr_matrix, linewidths=.01, annot = True, cmap='Blues')
#plt.savefig("Correlation Matrix.png") 
plt.show()

In [ ]:
#Correlation matrix sur UPDRS
corr_matrix['UPDRS'].sort_values(ascending=False)

A partir de la matrice de corrélation sur et de la projection sur notre label Y=UPDRS, on s'apperçoit que la variable la plus corélée et class information, mais dans notre cas on est pas entrain de faire une etude sur la classification mais c'est sur la regression, donc on vas retirer par la suite cette variable automatiquement. Donc y'a aucune corrélation de UPDRS avec les autres variables.

In [ ]:
sns.lmplot(x='Shimmer_APQ11',y='UPDRS',data=parkinson_DF,aspect=2,height=6)
plt.xlabel('Shimmer_APQ11: comme variable Independante')
plt.ylabel('UPDRS: comme variable Dependante')
plt.title('UPDRS Vs Shimmmer_APQ11')

Dans le graphique ci-dessus, nous adaptons la ligne de régression aux variables.

## Exploration des données

### Exploration uni-dimensionelle

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows = 9, ncols = 3,figsize=(10,15))

i_name = 0
            
for i in range(9):
    for j in range(3):
            i_name = i_name+1
            name_col = parkinson_DF.columns[i_name]
            axs[i, j].hist(parkinson_DF[name_col])
            axs[i, j].set_title(name_col)

fig.tight_layout()
#plt.savefig("Exploration uni dimensionelle avant transformation.png") 
plt.show()


#### Analyse sur Exploration uni-dimensionelle
D'apres le plot precedent, on peut voir que y'a plusieurs colonne ou leur parition ne sont pas repartis en loi normal ce qui nous invite à gerer çà en transformant nos données utilisant SQRT et LOG.


### Transformation du jeu de données

In [ ]:
#Suppression de quelques colonnes
#suppression de Class Information (classification des malades) pour faire la regression
parkinson_DF= parkinson_DF.drop('Class Information', axis=1)


In [ ]:
from math import sqrt, log

#Transformation des colonnes qui sont pas reparti en loi normal
parkinson_DF["LJitter_LOCAL"] = parkinson_DF["Jitter_LOCAL"].map(lambda x: log(x))
parkinson_DF["SJitter_LOCAL_ABSOLUTE"] = parkinson_DF["Jitter_LOCAL_ABSOLUTE"].map(lambda x: sqrt(x))
parkinson_DF["LJitter_RAP"] = parkinson_DF["Jitter_RAP"].map(lambda x: log(x))
parkinson_DF["LJitter_PPQ5"] = parkinson_DF["Jitter_PPQ5"].map(lambda x: log(x))
parkinson_DF["LJitter_DDP"] = parkinson_DF["Jitter_DDP"].map(lambda x: log(x))
parkinson_DF["LShimmer_LOCAL"] = parkinson_DF["Shimmer_LOCAL"].map(lambda x: log(x))
parkinson_DF["LShimmer_APQ3"] = parkinson_DF["Shimmer_APQ3"].map(lambda x: log(x))
parkinson_DF["LShimmer_APQ5"] = parkinson_DF["Shimmer_APQ5"].map(lambda x: log(x))
parkinson_DF["LShimmer_APQ11"] = parkinson_DF["Shimmer_APQ11"].map(lambda x: log(x))
parkinson_DF["LShimmer_DDA"] = parkinson_DF["Shimmer_DDA"].map(lambda x: log(x))
parkinson_DF["LStandard deviation"] = parkinson_DF["Standard deviation"].map(lambda x: log(x))
parkinson_DF["SNumber of pulses"] = parkinson_DF["Number of pulses"].map(lambda x: sqrt(x))
parkinson_DF["SNumber of periods"] = parkinson_DF["Number of periods"].map(lambda x: sqrt(x))
parkinson_DF["SStandard deviation of period"] = parkinson_DF["Standard deviation of period"].map(lambda x: sqrt(x))
parkinson_DF["SNumber of voice breaks"] = parkinson_DF["Number of voice breaks"].map(lambda x: sqrt(x))


#Suppression des anciennes colonnes
del parkinson_DF["Jitter_LOCAL"]
del parkinson_DF["Jitter_LOCAL_ABSOLUTE"]
del parkinson_DF["Jitter_RAP"]
del parkinson_DF["Jitter_PPQ5"]
del parkinson_DF["Jitter_DDP"]
del parkinson_DF["Shimmer_LOCAL"]
del parkinson_DF["Shimmer_APQ3"]
del parkinson_DF["Shimmer_APQ5"]
del parkinson_DF["Shimmer_APQ11"]
del parkinson_DF["Shimmer_DDA"]
del parkinson_DF["Standard deviation"]
del parkinson_DF["Number of pulses"]
del parkinson_DF["Number of periods"]
del parkinson_DF["Standard deviation of period"]
del parkinson_DF["Number of voice breaks"]


In [ ]:
parkinson_DF.head(5)

### Visualisation uni-dimensionelle apres modification

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows = 9, ncols = 3,figsize=(10,25))

i_name = 0
            
for i in range(9):
    for j in range(3):
            i_name = i_name+1
            name_col = parkinson_DF.columns[i_name]
            axs[i, j].hist(parkinson_DF[name_col])
            axs[i, j].set_title(name_col)

fig.tight_layout()
#plt.savefig("Exploration uni dimensionelle apres transformation.png") 
plt.show()


#### Analyse sur Exploration uni-dimensionelle apres modification
D'apres ce nouveau plot, on peut voir que les colonnes sont repartis en loi normalapres la transformation de nos données utilisant SQRT et LOG.



### Exploration multi-dimensionnelle

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(parkinson_DF, alpha=0.2, figsize=(35, 35), diagonal='kde')
plt.show()

###  Création de l'ensemble d'apprentissage / validation et de l'ensemble de test

In [ ]:
# Decoupage de dataset pour les features et label
X = parkinson_DF.drop('UPDRS', axis=1)
Y = parkinson_DF['UPDRS'].copy()

X.head()

In [ ]:
Y.head()

In [ ]:
#Phase de splittage des données

from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler  

X_1, X_2, Y_1, Y_2 = train_test_split(X, Y, train_size = 400, test_size = 200, random_state = 42)

X_av = X_1.to_numpy()
X_t = X_2.to_numpy()
Y_av = np.transpose([Y_1.to_numpy()])
Y_t = np.transpose([Y_2.to_numpy()])

In [ ]:
X.columns

**Remarque.** Scikit-learn fonctionne avec des numpy array. Nous allons donc utiliser les indices des colonnes:

0. SubjectId
1. Shimmer_LOCAL_DB
2. AC
3. NTH
4. HTN
5. Median pitch
6. Mean pitch
7. Minimum pitch
8. Maximum pitch
9. Mean period
11. Fraction of locally unvoiced frames
11. Degree of voice breaks
12. LJitter_LOCAL
13. SJitter_LOCAL_ABSOLUTE
14. LJitter_RAP
15. LJitter_PPQ5
16. LJitter_DDP
17. LShimmer_LOCAL 
18. LShimmer_APQ3
19. LShimmer_APQ5
20. LShimmer_APQ11
21. LShimmer_DDA
22. LStandard deviation
23. SNumber of pulses
24. SNumber of periods
25. SStandard deviation of period
26. SNumber of voice breaks


# Usage de la regression Lineaire avec ses differentes types et pénalisation



##  Régression linéaire univariée

### Relation unidimensionelle 

On étudie la qualité de la relation $\hat{Y} = X$, où $\hat{Y}$ est le vecteur de prédiction de l'UPDRS observée et $X$ le vecteur des prévisions de l'UPDRS observée calculée.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

fig, axs = plt.subplots(nrows = 1, ncols = 2,figsize=(15,10))

axs[0].plot(X_av[:,[10]], Y_av, 'o')
axs[0].plot([0, 60], [0, 100], 'r')
axs[0].set_xlabel('Fraction of locally unvoiced frames: $X^10 = \hat{Y}_{av}$')
axs[0].set_ylabel('UPDRS observée: $Y_{av}$')
axs[0].set_title('Regression lineaire univarie')


axs[1].plot(X_av[:,[10]], Y_av - X_av[:,[10]], 'o')
axs[1].plot([0, 300], [0, 0], 'r')
axs[1].set_xlabel('Fraction of locally unvoiced frames: $X^10= \hat{Y}_{av}$')
axs[1].set_ylabel('Résidus')
axs[1].set_title('Graphe des Résidus')

fig.tight_layout()
#plt.savefig("Analyse de Fraction of locally unvoiced frames avec UPDRS.png") 
plt.show()

print("Coefficient de détermination :", r2_score(Y_av, X_av[:,[10]]))
print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, X_av[:,[10]]))


A partir des resultats obtenus:<br>
- On peut voirque le R carre dans cette etude est négatif qui indique que le nombre de predictions est trop petit, et que le model est absolument pas bon, on peu meme compter à l'oeil nu le nombre de predictions.
- Sur le deuxieme plot on peut voir que le niveau de residus est tres élevé dans trop de valeurs "Y_av - x_10" sont loin d'etre egale à 0, avec une difference qui arrice jusqu'à -70, ce qui fait l'augmentation des residus et aussi la baisse de R carré


### Régression linéaire ordinaire 

On étudie la qualité de la relation $\hat{Y} = X\beta + \epsilon$, où $\hat{Y}$ est le vecteur de prédiction de l'UPDRS observée et $X$ le vecteur des prévisions de l'UPDRS observée calculée.

In [ ]:
from sklearn import linear_model

ols = linear_model.LinearRegression()
ols.fit(X_av[:,[16]], Y_av)
Y_hat_av = ols.predict(X_av[:,[16]])

print("Coefficient de régression : \n", ols.coef_)
print()

fig, axs = plt.subplots(nrows = 1, ncols = 2, sharex = True, figsize=(15,10))

axs[0].plot(X_av[:,[16]], Y_av, 'o')
axs[0].plot(X_av[:,[16]], Y_hat_av, 'r-')
#axs[0].set_ylim(0, 300)
axs[0].set_xlabel('Régression Ljitter_DDP: $\hat{Y}_{av}$')
axs[0].set_ylabel('UPDRS observée: $Y_{av}$')
axs[0].set_title('Regression lineaire ordinaire')

axs[1].plot(Y_hat_av, Y_av - Y_hat_av, 'o')
axs[1].plot([0, 300], [0, 0], 'r')
axs[1].set_xlabel('Prédiction Ljitter_DDP: $\hat{Y}_{av}$')
axs[1].set_ylabel('Résidus')
axs[1].set_title('Graphe des résidus')

fig.tight_layout()
#plt.savefig("Analyse LJitter_DDP avec UPDRS.png") 
plt.show()

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av))

A partir des resultats obtenus:<br>
- On peut voir que le R carre dans cette etude est trop petit 0.07, indiquant aisni que le model est pas bon et manque de précision, on peu meme compter à l'oeil nu le nombre de predictions.
- Sur le deuxieme plot on peut voir que le niveau de residus est tres élevé dans trop de valeurs "Y_av - Y_hat_av" sont loin d'etre egale à 0, avec une difference qui arrice jusqu'à 48, ce qui fait l'augmentation des residus et aussi la baisse de R carré



##  Régression linéaire multivariée

### Régression linéaire ordinaire



In [ ]:
#Application de la regression lineaire multivalué (𝑋𝑎𝑣,𝑌𝑎𝑣). 
from sklearn import linear_model

#creation de modele de regression lineaire
reg = linear_model.LinearRegression()

#entrainement de model
reg.fit(X_av, Y_av)

#affichage des coeffecients de regression pour le model
print("Coffecients de regression",reg.coef_)
print()

#prediction de model sur les deux sets X_av et X_t
Y_hat_av41 = reg.predict(X_av)
Y_hat_t41 = reg.predict(X_t)

print("****************Y_av **************************************")
print("Erreur quadratique (learning set) Y_av: %.2f" % mean_squared_error(Y_av, Y_hat_av41))
print("Coefficient de détermination (learning set) de Y_av: %.2f" % r2_score(Y_av, Y_hat_av41))
print("***********************************************************")
print()

print("****************Y_t ***********************************")
print("Erreur quadratique (testing set) Y_t: %.2f" % mean_squared_error(Y_t, Y_hat_t41))
print("Coefficient de détermination (testing set) de Y_t: %.2f" % r2_score(Y_t, Y_hat_t41))
print("*******************************************************")

print()

#affichage des coeffecients de regression 
coef = pd.Series(reg.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients pour regression lineaire multivalué")
#plt.savefig("CoeffecientsRegression Lineare multivalué.png") 
plt.show()


A partir de ce plot, on aperçoit que la variable la plus impactante dans cette regression lineaire multivalué sur ce dataset est cell ci: LShimmer_DDA. <br>
Et la valeur la moins impactante est LShimmer_APQ3

###  Régression linéaire pénalisée : Ridge


#### Modèle 1

In [ ]:
r_a01 = linear_model.Ridge(alpha = 0.1)
model_r_a01 = r_a01.fit(X_av, Y_av)
Y_hat_av_r01 = r_a01.predict(X_av)
Y_hat_t_r01 = r_a01.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r01))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r01))

coef = pd.Series(model_r_a01.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 0.1")
plt.show()

####### Modele 1 <br>
A partir de plot du modele 1, on aperçoit que la variable la plus impactante dans cette regression lineaire multivalué pénalisé avec Ridge d'un facteur alpha= 0.1 sur ce dataset est cell ci: "NTH". <br>
Et la valeur la moins impactante est "Snumber of pulses"

#### Modèle 2

In [ ]:
r_a1 = linear_model.Ridge(alpha = 1)
model_r_a1 = r_a1.fit(X_av, Y_av)
Y_hat_av_r1 = r_a1.predict(X_av)
Y_hat_t_r1 = r_a1.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r1))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r1))

coef = pd.Series(model_r_a1.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 1")
plt.show()

####### Modele 2 <br>
A partir de plot du modele 2, on aperçoit que la variable la plus impactante dans cette regression lineaire multivalué pénalisé avec Ridge d'un facteur alpha= 1 sur ce dataset est cell ci: "NTH". <br>
Et la valeur la moins impactante est "LShimmer_LOCAL"

#### Modèle 3

In [ ]:
r_a10 = linear_model.Ridge(alpha = 10)
model_r_a10 = r_a10.fit(X_av, Y_av)
Y_hat_av_r10 = r_a10.predict(X_av)
Y_hat_t_r10 = r_a10.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r10))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r10))

coef = pd.Series(model_r_a10.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 10")
plt.show()

####### Modele 3 <br>
A partir de plot du modele 3, on aperçoit que la variable la plus impactante dans cette regression lineaire multivalué pénalisé avec Ridge d'un facteur alpha= 10 sur ce dataset est cell ci: "LJitter_PPQ5". <br>
Et la valeur la moins impactante est "Snumber of voice breaks"

#### Visualisation globale de l'évolution des coefficients Ridge Linear Regression en fonction de alpha 

In [ ]:
# Visualisez globalement l'évolution des coefficients en fonction de alpha

import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

# #############################################################################
# Compute paths

n_alphas = 200
alphas = np.logspace(5, 0, n_alphas)

coefs = []
for a in alphas:
    ridge = linear_model.Ridge(alpha=a, fit_intercept=False)
    ridge.fit(X_t, Y_t)
    coefs.append(ridge.coef_)

# #############################################################################
# Display results

ax = plt.gca()

coefs = np.array(coefs)
coefs = coefs.reshape(200,27)

ax.plot(alphas, coefs)
ax.set_xscale("log")
ax.set_xlim(ax.get_xlim()[::-1])  #  reversed axis
plt.xlabel("alpha")
plt.ylabel("Poids")
plt.title("Ridge coefficients as a function of the L2 regularization")
plt.axis("tight")
#plt.savefig("Ridge Coeffecients Regression Lineare.png") 
plt.show()

On peut appercevoir que le plus que alpha augmente plus les poids des coefficients tendent vers zero, et reciproquement ils devirgent de zero.

##### Ridge Linear Regression Best Alpha detection with RidgeCV

In [ ]:
#Mettez en place une procédure d'apprentissage rigoureuse pour trouver le paramètre alpha optimal de la régression ridge. 
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge

ridgeCV = RidgeCV(alphas=[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1,10]).fit(X_av, Y_av)

print("Le score 'R carre' de la cross validation de Ridge Linear Regression model", ridgeCV.score(X_av, Y_av))
print("Les coeffecients du model RidgeCV Linear Regression sont:",ridgeCV.coef_)
print()
#print(ridgeCV.intercept_)
print("Le parametre alpha optimal pour la regression ridge est alpha = ",ridgeCV.alpha_)


from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score


ridge_cv=RidgeCV(alphas=[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1,10], store_cv_values=True)
ridge_mod = ridge_cv.fit(X_av,Y_av)
ypred = ridge_mod.predict(X_av)
print("L'Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, ypred))


x_ax = range(len(X_av))
plt.title("Comparaison des resultats Y_hat_av (Prediction) avec Y(original) avec alpha = 10.0") 
plt.scatter(x_ax,Y_av , s=10, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
#plt.savefig("Ridge Comparasion des valeurs existantes et valeurs predites.png") 
plt.show()


A partir de ces resultats, dans ce dataset avec la technique de RidgeCV, on voit que le alpha le plus otimale est alpha=10.<br>
Sur le plot suivant on peut voir la comparaison entre les valeurs initiales de UPDRS celles predites et celles non predites, ou chaque ligne rouge qui passe sur un point bleu indique une prediction et sinon cette valeur n'est pas predite.

### Régression linéaire pénalisée : Lasso


In [ ]:
# difference of lasso and ridge regression is that some of the coefficients can be zero i.e. some of the features are 
# completely neglected
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression

lasso = Lasso()
lasso.fit(X_av,Y_av)

lasso01 = Lasso(alpha=0.1, max_iter=10e5)
lasso01.fit(X_av,Y_av)

lasso001 = Lasso(alpha=0.01, max_iter=10e5)
lasso001.fit(X_av,Y_av)

lasso00001 = Lasso(alpha=0.0001, max_iter=10e5)
lasso00001.fit(X_av,Y_av)

lr = LinearRegression()
lr.fit(X_av,Y_av)

plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
plt.plot(lasso01.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Lasso; $\alpha = 0.1$') # alpha here is for transparency
plt.plot(lasso001.coef_,alpha=0.7,linestyle='none',marker='o',markersize=5,color='green',label=r'Lasso; $\alpha = 0.01$',zorder=7) # alpha here is for transparency

plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=1)
plt.title("Régression avec Lasso et dépendance de la sélection de coefficients à la valeur du alpha")
#plt.savefig("Lasso coeff regression lineaire.png") 

plt.show()


##### Lasso Linear Regression Best Alpha detection with LassoCV

In [ ]:
# Mettez en place une procédure d'apprentissage rigoureuse pour trouver le paramètre alpha optimal dans la régression lasso.
from sklearn.linear_model import LassoCV
from sklearn.datasets import make_regression

lassocv = LassoCV(alphas=[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.2,0.5, 1,10]).fit(X_av, Y_av)
print("Le score 'R carre' de la cross validation de Lasso Linear Regression model", lassocv.score(X_av, Y_av))
print("Les coeffecients du model LassoCV Linear Regression sont:",lassocv.coef_)
print()
print("Le parametre alpha optimal pour la regression Lasso est alpha = ",lassocv.alpha_)


from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score


lasso_cv=LassoCV(alphas=[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.2,0.5, 1,10])
lasso_mod = lasso_cv.fit(X_av,Y_av)
ypred = lasso_mod.predict(X_av)
print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, ypred))


x_ax = range(len(X_av))
plt.title("Comparaison des resultats Y_hat_av(prediction) avec Y(original) avec alpha = 0.1") 
plt.scatter(x_ax,Y_av , s=10, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
#plt.savefig("Lasso Comparasion des valeurs existantes et valeurs predites.png") 

plt.legend()
plt.show()

A partir de ces resultats, dans ce dataset avec la technique de LassoCV, on voit que le alpha le plus otimale est alpha=0.1.<br>
Sur le plot suivant on peut voir la comparaison entre les valeurs initiales de UPDRS celles predites et celles non predites, ou chaque ligne rouge qui passe sur un point bleu indique une prediction et sinon cette valeur n'est pas predite.

# Ridge vs Lasso with cross_val_score

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge, Lasso

plt.figure(figsize=(5, 3))
alphas = np.logspace(-3, -1, 30)
for Model in [Lasso, Ridge]:
    scores = [cross_val_score(Model(alpha), X_av, Y_av, cv=3).mean()
            for alpha in alphas]
    plt.plot(alphas, scores, label=Model.__name__)

plt.legend(loc='lower right')
plt.xlabel('alpha')
plt.ylabel('cross validation score')
plt.title('Variation des scores de RIDGE et LASSO par rapport à alpha')
plt.tight_layout()
#plt.savefig("Lasso vs ridge vartion scores.png") 
plt.show()



A partir de ces resultats, dans ce dataset avec la technique de cross_val_score sur les deux modeles Ridge et Lasso, on voit que plus le alpha tend vers zero, la precesion des deux modeles aussi converge vers zero, mais en montant de zero vers 0.1, la precesion du modele Lasso monte en exponentiel tant que la precision de modele Ridge monte d'une façon moins exponentionnel par rapport à Lasso. Et vers la valeur alpha = 0.1 converge ensemble, et Lasso garde toujours le dessus sur Ridge<br>


# Remarques:
 On faisant ce TP, j'ai remarqué que l'inclusion de la colonne 'Class Information' a fait augmenter les performances de tout les modeles utilisées (Multivariate Linear Regression, Ridge Linear Regression et Lasso Linear Regression) au lieu d'avoir des R carre de 0.43, la performance a augmenté vers 0.62 environ.<br>
 J'ai preferé de garder le TP comme çà car je suis persuadé que (Class Information) est utile pour la classification et non pas la regression et c'est la raison à laquelle je l'ai pas prise.